In [1]:
#Encoding scripts/ compressing scripts

import os
import torch
import numpy as np
import util
import rans
from torch_vae.tvae_beta_binomial import BetaBinomialVAE
from torch_vae import tvae_utils
from torchvision import datasets, transforms
import time
from PIL import Image



# from torchvision

# our_transforms = transforms.Compose([transforms.PILToTensor(), transforms.Resize((28,28))])
# images = []
# for img in os.listdir('new_dataset'):
#     images.append(our_transforms(Image.open(f'new_dataset/{img}')))

# images = torch.stack(images)
# print(images.shape)

rng = np.random.RandomState(0)
np.seterr(over='raise')

prior_precision = 8
obs_precision = 14
q_precision = 14

num_images = 1000

compress_lengths = []

latent_dim = 50
latent_shape = (1, latent_dim)
model = BetaBinomialVAE(hidden_dim=200, latent_dim=latent_dim)
model.load_state_dict(
    torch.load('saved_params/torch_vae_beta_binomial_params',
               map_location=lambda storage, location: storage))
model.eval()

rec_net = tvae_utils.torch_fun_to_numpy_fun(model.encode)
gen_net = tvae_utils.torch_fun_to_numpy_fun(model.decode)

obs_append = tvae_utils.beta_binomial_obs_append(255, obs_precision)
obs_pop = tvae_utils.beta_binomial_obs_pop(255, obs_precision)

vae_append = util.vae_append(latent_shape, gen_net, rec_net, obs_append,
                             prior_precision, q_precision)
vae_pop = util.vae_pop(latent_shape, gen_net, rec_net, obs_pop,
                       prior_precision, q_precision)

# load some mnist images
mnist = datasets.MNIST('data/mnist', train=False, download=True,
                       transform=transforms.Compose([transforms.ToTensor()]))
images = mnist.test_data[:num_images]

transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Resize to 28x28
    transforms.ToTensor()         # Convert to tensor
])

images = [image.float().view(1, -1) for image in images]

# randomly generate some 'other' bits
other_bits = rng.randint(low=1 << 16, high=1 << 31, size=50, dtype=np.uint32)
state = rans.unflatten(other_bits)


print_interval = 10
encode_start_time = time.time()
for i, image in enumerate(images):
    state = vae_append(state, image)

    if not i % print_interval:
        print('Encoded {}'.format(i))

    compressed_length = 32 * (len(rans.flatten(state)) - len(other_bits)) / (i+1)
    compress_lengths.append(compressed_length)

print('\nAll encoded in {:.2f}s'.format(time.time() - encode_start_time))
compressed_message = rans.flatten(state)
with open('compressed_image/compressed_message.bin', 'wb') as f:
    np.array(compressed_message, dtype=np.uint32).tofile(f)

print("Compressed message saved to 'compressed_message.bin'.")
compressed_bits = 32 * (len(compressed_message) - len(other_bits))
print("Used " + str(compressed_bits) + " bits.")
print('This is {:.2f} bits per pixel'.format(compressed_bits
                                             / (num_images * 784)))

np.savetxt('compressed_image/compressed_lengths_cts', np.array(compress_lengths))


/tmp/ipykernel_32428/3150271421.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load('saved_params/torch_vae_beta_binomial_params',
/home/pradeep/anaconda3/envs/d

Encoded 0
Encoded 10
Encoded 20
Encoded 30
Encoded 40
Encoded 50
Encoded 60
Encoded 70
Encoded 80
Encoded 90
Encoded 100
Encoded 110
Encoded 120
Encoded 130
Encoded 140
Encoded 150
Encoded 160
Encoded 170
Encoded 180
Encoded 190
Encoded 200
Encoded 210
Encoded 220
Encoded 230
Encoded 240
Encoded 250
Encoded 260
Encoded 270
Encoded 280
Encoded 290
Encoded 300
Encoded 310
Encoded 320
Encoded 330
Encoded 340
Encoded 350
Encoded 360
Encoded 370
Encoded 380
Encoded 390
Encoded 400
Encoded 410
Encoded 420
Encoded 430
Encoded 440
Encoded 450
Encoded 460
Encoded 470
Encoded 480
Encoded 490
Encoded 500
Encoded 510
Encoded 520
Encoded 530
Encoded 540
Encoded 550
Encoded 560
Encoded 570
Encoded 580
Encoded 590
Encoded 600
Encoded 610
Encoded 620
Encoded 630
Encoded 640
Encoded 650
Encoded 660
Encoded 670
Encoded 680
Encoded 690
Encoded 700
Encoded 710
Encoded 720
Encoded 730
Encoded 740
Encoded 750
Encoded 760
Encoded 770
Encoded 780
Encoded 790
Encoded 800
Encoded 810
Encoded 820
Encoded 830
Enc